In [1]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Get the OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
MILVUS_HOST = "localhost"
MILVUS_PORT = "19530"

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Milvus
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

loader = TextLoader(
    "../data/Robinson_Advisory.txt", 
    encoding="windows-1252"
)

docs = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
docs = text_splitter.split_documents(docs)


Created a chunk of size 1567, which is longer than the specified 1024
Created a chunk of size 1075, which is longer than the specified 1024
Created a chunk of size 2084, which is longer than the specified 1024
Created a chunk of size 1331, which is longer than the specified 1024
Created a chunk of size 1177, which is longer than the specified 1024


In [4]:
embeddings = OpenAIEmbeddings()

vector_store = Milvus.from_documents(
    docs,
    embedding=embeddings,
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT}
)


/root/Contract-Advisor-RAG-/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [5]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI


chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="map_rerank")
query = "Can the Agreement or any of its obligations be assigned?"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)


/root/Contract-Advisor-RAG-/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/root/Contract-Advisor-RAG-/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/root/Contract-Advisor-RAG-/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'output_text': ' Yes, the Agreement or any of its obligations can be assigned.'}

In [6]:
eval_questions = [
    {
        "query": "Who are the parties to the Agreement and what are their defined names?",
        "answer": "Cloud Investments Ltd. (“Company”) and Jack Robinson (“Advisor”)"
    },
    {
        "query": "What is the termination notice?",
        "answer": "According to section 4:14 days for convenience by both parties. The Company may terminate without notice if the Advisor refuses or cannot perform the Services or is in breach of any provision of this Agreement."
    },
    {
        "query": "What are the payments to the Advisor under the Agreement?",
        "answer": "According to section 6: 1. Fees of $9 per hour up to a monthly limit of $1,500, 2. Workspace expense of $100 per month, 3. Other reasonable and actual expenses if approved by the company in writing and in advance."
    },
    {
        "query": "Can the Agreement or any of its obligations be assigned?",
        "answer": "Under section 1.1 the Advisor can’t assign any of his obligations without the prior written consent of the Company, 2. Under section 9 the Advisor may not assign the Agreement and the Company may assign it, 3 Under section 9 of the Undertaking the Company may assign the Undertaking."
    },
    {
        "query": "Who owns the IP?",
        "answer": "According to section 4 of the Undertaking (Appendix A), Any Work Product, upon creation, shall be fully and exclusively owned by the Company."
    },
    {
        "query": " Can the Advisor charge for meal time?",
        "answer": "No. See Section 6.1, Billable Hour doesn’t include meals or travel time."
    },
    {
        "query": " Is there a non-compete obligation to the Advisor?",
        "answer": "Yes. During the term of engagement with the Company and for a period of 12 months thereafter."
    },
    {
        "query": "In which street does the Advisor live?",
        "answer": "1 Rabin st, Tel Aviv, Israel"
    },
    {
        "query": "Is the Advisor entitled to social benefits?",
        "answer": "No. According to section 8 of the Agreement, the Advisor is an independent consultant and shall not be entitled to any overtime pay, insurance, paid vacation, severance payments or similar fringe or employment benefits from the Company."
    },
    {
        "query": "What happens if the Advisor claims compensation based on employment relationship with the Company?",
        "answer": "If the Advisor is determined to be an employee of the Company by a governmental authority, payments to the Advisor will be retroactively reduced so that 60% constitutes salary payments and 40% constitutes payment for statutory rights and benefits. The Company may offset any amounts due to the Advisor from any amounts payable under the Agreement. The Advisor must indemnify the Company for any losses or expenses incurred if an employer/employee relationship is determined to exist."
    },
]

In [12]:
result = chain({"input_documents": docs, "question": eval_questions[0]["query"]}, return_only_outputs=True)
result

/root/Contract-Advisor-RAG-/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'output_text': ' The parties to the Agreement are Cloud Investments Ltd. and Mr. Jack Robinson. Their defined names are "Company" and "Advisor" respectively.'}

In [13]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory

llm = OpenAI(temperature=0)
# Notice that "chat_history" is present in the prompt template
template = """You are a chatbot designed to interpreate a contract and have a conversation with a human about the contract.

Previous conversation:
{chat_history}

New human question: {question}
Response:"""
prompt = PromptTemplate.from_template(template)
# Notice that we need to align the `memory_key`
memory = ConversationBufferWindowMemory(memory_key="chat_history", k=1)   
# memory = ConversationBufferMemory(memory_key="chat_history")
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [15]:
# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "My name is Abel"})



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot designed to interpreate a contract and have a conversation with a human about the contract.

Previous conversation:
Human: hi
AI:  Hello! I am a chatbot designed to interpret and discuss contracts. How can I assist you?

New human question: My name is Abel
Response:

> Finished chain.


{'question': 'My name is Abel',
 'chat_history': 'Human: hi\nAI:  Hello! I am a chatbot designed to interpret and discuss contracts. How can I assist you?',
 'text': ' Nice to meet you, Abel! How can I help you with your contract?'}

In [16]:
conversation({"question": "What is the termination notice?"})



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot designed to interpreate a contract and have a conversation with a human about the contract.

Previous conversation:
Human: hi
AI:  Hello! I am a chatbot designed to interpret and discuss contracts. How can I assist you?
Human: My name is Abel
AI:  Nice to meet you, Abel! How can I help you with your contract?

New human question: What is the termination notice?
Response:

> Finished chain.


{'question': 'What is the termination notice?',
 'chat_history': 'Human: hi\nAI:  Hello! I am a chatbot designed to interpret and discuss contracts. How can I assist you?\nHuman: My name is Abel\nAI:  Nice to meet you, Abel! How can I help you with your contract?',
 'text': ' The termination notice is a clause in the contract that outlines the process and requirements for ending the agreement between the parties involved. It typically includes the amount of notice that must be given, the reasons for termination, and any penalties or consequences for terminating the contract early. Is there anything specific you would like to know about the termination notice in your contract?'}

In [17]:
conversation({"question": "What is my name?"})



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot designed to interpreate a contract and have a conversation with a human about the contract.

Previous conversation:
Human: hi
AI:  Hello! I am a chatbot designed to interpret and discuss contracts. How can I assist you?
Human: My name is Abel
AI:  Nice to meet you, Abel! How can I help you with your contract?
Human: What is the termination notice?
AI:  The termination notice is a clause in the contract that outlines the process and requirements for ending the agreement between the parties involved. It typically includes the amount of notice that must be given, the reasons for termination, and any penalties or consequences for terminating the contract early. Is there anything specific you would like to know about the termination notice in your contract?

New human question: What is my name?
Response:

> Finished chain.


{'question': 'What is my name?',
 'chat_history': 'Human: hi\nAI:  Hello! I am a chatbot designed to interpret and discuss contracts. How can I assist you?\nHuman: My name is Abel\nAI:  Nice to meet you, Abel! How can I help you with your contract?\nHuman: What is the termination notice?\nAI:  The termination notice is a clause in the contract that outlines the process and requirements for ending the agreement between the parties involved. It typically includes the amount of notice that must be given, the reasons for termination, and any penalties or consequences for terminating the contract early. Is there anything specific you would like to know about the termination notice in your contract?',
 'text': ' Your name is Abel, as you mentioned earlier. Is there anything else you would like to know about your contract?'}